In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score  
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict 
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import confusion_matrix 
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sklearn as skl
warnings.filterwarnings('ignore')
%matplotlib inline

In [19]:
df = pd.read_csv('Columbia.csv', index_col=0) 
#Remember to change this part to the directory of where your csv file is, otherwise it won't work

In [20]:
df.head()

,member_id,CLI,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
id,,,,,,,,,,,,,,,,,,,,,
1077501,1296599,5000,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1077430,1314167,2500,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1077175,1313524,2400,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1076863,1277178,10000,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1075358,1311748,3000,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
#Removes the NaN values per columns, which is what the axis=1 is for
df = df.dropna(axis=1)
df.head()

,member_id,CLI,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,...,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq
id,,,,,,,,,,,,,,,,,,,,,
1077501,1296599,5000,36 months,10.65,162.87,B,B2,10+ years,RENT,24000.0,...,861.07,0.00,0.00,0.00,171.62,16-Jan,0,1,INDIVIDUAL,0
1077430,1314167,2500,60 months,15.27,59.83,C,C4,< 1 year,RENT,30000.0,...,435.17,0.00,117.08,1.11,119.66,13-Sep,0,1,INDIVIDUAL,0
1077175,1313524,2400,36 months,15.96,84.33,C,C5,10+ years,RENT,12252.0,...,603.65,0.00,0.00,0.00,649.91,16-Jan,0,1,INDIVIDUAL,0
1076863,1277178,10000,36 months,13.49,339.31,C,C1,10+ years,RENT,49200.0,...,2209.33,16.97,0.00,0.00,357.48,15-Jan,0,1,INDIVIDUAL,0
1075358,1311748,3000,60 months,12.69,67.79,B,B5,1 year,RENT,80000.0,...,1009.07,0.00,0.00,0.00,67.79,16-Jan,0,1,INDIVIDUAL,0


In [22]:
cols_to_drop = ['policy_code', 'initial_list_status', 'collections_12_mths_ex_med', 'pymnt_plan', 'application_type', 'acc_now_delinq', 'purpose', 'title', 'zip_code', 'addr_state']
df.drop(cols_to_drop, axis=1, inplace=True)
df.head()

,member_id,CLI,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,...,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_credit_pull_d
id,,,,,,,,,,,,,,,,,,,,,
1077501,1296599,5000,36 months,10.65,162.87,B,B2,10+ years,RENT,24000.0,...,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,171.62,16-Jan
1077430,1314167,2500,60 months,15.27,59.83,C,C4,< 1 year,RENT,30000.0,...,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,119.66,13-Sep
1077175,1313524,2400,36 months,15.96,84.33,C,C5,10+ years,RENT,12252.0,...,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,649.91,16-Jan
1076863,1277178,10000,36 months,13.49,339.31,C,C1,10+ years,RENT,49200.0,...,0.0,12226.302210,12226.30,10000.00,2209.33,16.97,0.00,0.00,357.48,15-Jan
1075358,1311748,3000,60 months,12.69,67.79,B,B5,1 year,RENT,80000.0,...,766.9,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,67.79,16-Jan


In [23]:
df.shape

(100, 32)

In [31]:
cols = df.columns
train_df = df
train_df = pd.get_dummies(train_df, columns=cols)
train_df.head()

,member_id_1261745,member_id_1269083,member_id_1272024,member_id_1273773,member_id_1274517,member_id_1276120,member_id_1277178,member_id_1277395,member_id_1278095,member_id_1278806,...,last_credit_pull_d_15-Apr,last_credit_pull_d_15-Dec,last_credit_pull_d_15-Jan,last_credit_pull_d_15-Jul,last_credit_pull_d_15-Jun,last_credit_pull_d_15-Mar,last_credit_pull_d_15-Nov,last_credit_pull_d_15-Oct,last_credit_pull_d_15-Sep,last_credit_pull_d_16-Jan
id,,,,,,,,,,,,,,,,,,,,,
1077501,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1077430,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1077175,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1076863,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1075358,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [37]:
all_features = train_df.drop("CLI",axis=1)
Targeted_feature = train_df["CLI"]

# total of 6891 records in dataset
# Divide the data set into two- 70% for train and 30% for test

#X_train,X_test,y_train,y_test = train_test_split(all_features,Targeted_feature,test_size=0.3)  
X_train,X_test,y_train,y_test = train_test_split(all_features,Targeted_feature,test_size=0.3,random_state=42) 

# Check the train test data set shape
X_train.shape,X_test.shape,y_train.shape,y_test.shape

KeyError: "['CLI'] not found in axis"

In [38]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 
label = LabelEncoder()
loan_status = ["Charged Off", "Current", "Fully Paid"]

int_data = label.fit_transform(loan_status)
int_data = int_data.reshape(len(int_data), 1)

onehot_data = OneHotEncoder(sparse=False)
onehot_data = onehot_data.fit_transform(int_data)
print("Categorical data encoded into integer values....\n")
print(onehot_data)

Categorical data encoded into integer values....

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [39]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

ValueError: could not convert string to float: ' 36 months'

In [40]:
train_df.columns

Index(['member_id_1261745', 'member_id_1269083', 'member_id_1272024',
       'member_id_1273773', 'member_id_1274517', 'member_id_1276120',
       'member_id_1277178', 'member_id_1277395', 'member_id_1278095',
       'member_id_1278806',
       ...
       'last_credit_pull_d_15-Apr', 'last_credit_pull_d_15-Dec',
       'last_credit_pull_d_15-Jan', 'last_credit_pull_d_15-Jul',
       'last_credit_pull_d_15-Jun', 'last_credit_pull_d_15-Mar',
       'last_credit_pull_d_15-Nov', 'last_credit_pull_d_15-Oct',
       'last_credit_pull_d_15-Sep', 'last_credit_pull_d_16-Jan'],
      dtype='object', length=1374)